# Decoder Transformers with PyTorch

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

In [5]:
token_to_id = {
    'what': 0,
    'is': 1,
    'kael': 2,
    'awesome': 3,
    '<EOS>': 4,
}

id_to_token = dict(map(reversed, token_to_id.items()))

inputs = torch.tensor([[token_to_id['what'],
                        token_to_id['is'],
                        token_to_id['kael'],
                        token_to_id['<EOS>'],
                        token_to_id['awesome']],
                        
                        [token_to_id['kael'],
                         token_to_id['is'],
                         token_to_id['what'],
                         token_to_id['<EOS>'],
                         token_to_id['awesome']]])

labels = torch.tensor([[token_to_id['is'],
                        token_to_id['kael'],
                        token_to_id['<EOS>'],
                        token_to_id['awesome'],
                        token_to_id['<EOS>']],
                        
                        [token_to_id['is'],
                         token_to_id['what'],
                         token_to_id['<EOS>'],
                         token_to_id['awesome'],
                         token_to_id['<EOS>']]])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [9]:
class PositionEncoding(nn.Module):
    def __init__(self, d_model=2, max_len=6):
        super().__init__()

        pe = torch.zeros(max_len, d_model)

        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)

        embedding_index = torch.arange(start=0, end=d_model, step=2).float()

        div_term = 1 / torch.tensor(10000.)**(embedding_index / d_model)

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)
    
    def forward(self, word_embeddings):
        return word_embeddings + self.pe[:word_embeddings.size(0)]

In [10]:
class Attention(nn.Module):
    def __init__(self, d_model=2):
        super().__init__()

        self.d_model = d_model

        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = 0
        self.col_dim = 1


    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):

        q = self.W_q(encodings_for_q)
        k = self.W_k(encodings_for_k)
        v = self.W_v(encodings_for_v)

        # ((q * k^T) / sqrt(d_model)) * v

        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=1e-9)

        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)

        attention_scores = torch.matmul(attention_percents, v)

        return attention_scores

In [19]:
seed = 45
class DecoderTransformer(nn.Module):
    def __init__(self, num_tokens=4, d_model=2, max_len=6):
        super().__init__()

        torch.manual_seed(seed=seed)

        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)

        self.pe = PositionEncoding(d_model=d_model, max_len=max_len)

        self.self_attention = Attention(d_model=d_model)

        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):
        word_embeddings = self.we(token_ids)
        position_encoded = self.pe(word_embeddings)


        mask = torch.tril(torch.ones((token_ids.size(0), token_ids.size(0))))

        mask = mask == 0

        self_attention_values = self.self_attention(position_encoded, position_encoded, position_encoded, mask=mask)

        residual_connection_values = position_encoded + self_attention_values

        fc_layer_output = self.fc_layer(residual_connection_values)

        return fc_layer_output
    
    def training_step(self, batch, batch_idx, optimizer):
        input_tokens, labels = batch
        output = self(input_tokens[0])
        loss = self.loss(output, labels[0])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        return loss

In [35]:
model = DecoderTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)

model_input = torch.tensor([token_to_id['what'],
                            token_to_id['is'],
                            token_to_id['kael'],
                            token_to_id['<EOS>']])

def predict(model_input):
    
    model.eval()

    input_length = model_input.size(0)

    predictions = model(model_input)

    predicted_id = torch.tensor([torch.argmax(predictions[-1, :])])


    predicted_ids = predicted_id

    max_length = 6

    for i in range(input_length, max_length):
        if(predicted_id == torch.tensor([token_to_id['<EOS>']])):
            break

        model_input = torch.cat((model_input, predicted_id))

        predictions = model(model_input)
        predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
        predicted_ids = torch.cat((predicted_ids, predicted_id))


    for id in predicted_ids:
        print(f'\t {id_to_token[id.item()]}')

In [36]:
print('Predicted tokens before training:')
predict(model_input)

Predicted tokens before training:
	 awesome
	 awesome
	 what


In [37]:
epochs = 10

optimizer = Adam(model.parameters(), lr=0.1)

model.train()

for epoch in range(epochs):
    total_loss = 0
    for batch_idx, batch in enumerate(dataloader):
        loss = model.training_step(batch, batch_idx, optimizer)
        total_loss += loss

    print(f'Epoch: {epoch + 1} / {epochs} | Loss: {total_loss}')
        

Epoch: 1 / 10 | Loss: 3.431887149810791
Epoch: 2 / 10 | Loss: 2.6273903846740723
Epoch: 3 / 10 | Loss: 2.1565098762512207
Epoch: 4 / 10 | Loss: 1.8088533878326416
Epoch: 5 / 10 | Loss: 1.4911552667617798
Epoch: 6 / 10 | Loss: 1.175260066986084
Epoch: 7 / 10 | Loss: 0.9104108810424805
Epoch: 8 / 10 | Loss: 0.722602128982544
Epoch: 9 / 10 | Loss: 0.5688287019729614
Epoch: 10 / 10 | Loss: 0.44615641236305237


In [39]:
print('Predicted tokens after training:')
predict(model_input)

Predicted tokens after training:
	 awesome
	 <EOS>


In [40]:
model_input = torch.tensor([token_to_id['kael'],
                            token_to_id['is'],
                            token_to_id['what'],
                            token_to_id['<EOS>']])

print('Predicted tokens after training:')
predict(model_input)

Predicted tokens after training:
	 awesome
	 <EOS>
